In [ ]:
import camelot
import pandas as pd
import os

# show full cells text
pd.set_option('display.max_colwidth', None)

PATH = './bills/'
month = {'Январь': 1, 'Февраль': 2, 'Март': 3, 'Апрель': 4, 'Май': 5, 'Июнь': 6, \
         'Июль': 7, 'Август': 8, 'Сентябрь': 9, 'Октябрь': 10, 'Ноябрь': 11, 'Декабрь': 12}


with open('checked_files.txt', 'r+') as checked_files:
    files = checked_files.readlines()
    for file in os.listdir(PATH):
        if '.pdf' in file and file+'\n' not in files:
            
            # scan pdf to tables
            tables = camelot.read_pdf(PATH+file, strip_text='\n')

            # main data
            df2 = tables[2].df

            # calculate slices
            range1_start = df2.index[df2[0].str.contains("Тех.обсл")][0]
            range1_stop = df2.index[df2[0].str.contains("Домофон")][0]
            range2_start = df2.index[df2[0].str.contains("Холодное")][0]
            range2_stop = df2.index[df2[0].str.contains("Итого к оплате за расчетный период")][0]

            # create df_month
            df_month = pd.concat([df2.iloc[range1_start:range1_stop+1], df2.iloc[range2_start:range2_stop]])
            df_month.columns = df2.iloc[0]

            # rename some values
            df_month.iloc[:,2:15] = df_month.iloc[:,2:15].replace('Х', 0)
            df_month.iloc[:,1:2] = df_month.iloc[:,1:2].replace(['Х', 'кВт'], ['кВт*ч', 'кВт*ч'])
            df_month.iloc[:,:1] = df_month.iloc[:,:1].replace(['Электроэнергия:', '- дн. начисление', \
                '- нч. начисление'], ['Электроэнергия', 'Электроэнергия день', 'Электроэнергия ночь'])

            # calculate period
            for index, table in enumerate(tables):
                if table.df.shape == (5, 2):
                    period = table.df[0][0]
                    period = period[period.find(')')+1:period.find(' г')].split()
                    period = period[-2:]
                    period[0] = month[period[0]]
                    df_month[['Месяц', 'Год']] = period
            
            df_month.to_csv('bills.csv', mode='a', index=False, header=False, sep=';', encoding='windows-1251')
            checked_files.write(file+'\n')
            print(file+' преобразован')